<a href="https://colab.research.google.com/github/RhonanBangay/CPEN-EXE-LAB/blob/main/Question_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.datasets import load_breast_cancer
from sklearn.svm import SVR
  
can = load_breast_cancer()
  
# The data set is presented in a dictionary form:
print(can.keys())

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module'])


In [34]:
d_data = pd.DataFrame(can['data'],
                       columns = can['feature_names'])
  
# cancer column is our target
df_target = pd.DataFrame(can['target'], 
                     columns =['Can'])
  
print("Feature Variables: ")
print(d_data.info())

Feature Variables: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 30 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   mean radius              569 non-null    float64
 1   mean texture             569 non-null    float64
 2   mean perimeter           569 non-null    float64
 3   mean area                569 non-null    float64
 4   mean smoothness          569 non-null    float64
 5   mean compactness         569 non-null    float64
 6   mean concavity           569 non-null    float64
 7   mean concave points      569 non-null    float64
 8   mean symmetry            569 non-null    float64
 9   mean fractal dimension   569 non-null    float64
 10  radius error             569 non-null    float64
 11  texture error            569 non-null    float64
 12  perimeter error          569 non-null    float64
 13  area error               569 non-null    float64
 14  smooth

In [53]:

print("Dataframe: ")
print(d_data.head())

Dataframe: 
   mean radius  mean texture  mean perimeter  mean area  mean smoothness  \
0        17.99         10.38          122.80     1001.0          0.11840   
1        20.57         17.77          132.90     1326.0          0.08474   
2        19.69         21.25          130.00     1203.0          0.10960   
3        11.42         20.38           77.58      386.1          0.14250   
4        20.29         14.34          135.10     1297.0          0.10030   

   mean compactness  mean concavity  mean concave points  mean symmetry  \
0           0.27760          0.3001              0.14710         0.2419   
1           0.07864          0.0869              0.07017         0.1812   
2           0.15990          0.1974              0.12790         0.2069   
3           0.28390          0.2414              0.10520         0.2597   
4           0.13280          0.1980              0.10430         0.1809   

   mean fractal dimension  ...  worst radius  worst texture  worst perimeter  \


In [36]:
from sklearn.model_selection import train_test_split
  
X_train, X_test, y_train, y_test = train_test_split(
                        d_data, np.ravel(df_target),
                test_size = 0.30, random_state = 101)

In [39]:

# train the model on train set
model = SVC()
model.fit(X_train, y_train)
  
# print prediction results
predictions = model.predict(X_test)
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.95      0.85      0.90        66
           1       0.91      0.97      0.94       105

    accuracy                           0.92       171
   macro avg       0.93      0.91      0.92       171
weighted avg       0.93      0.92      0.92       171



In [54]:

from sklearn.model_selection import GridSearchCV
from sklearn.svm import LinearSVR
from sklearn.svm import SVR
  
# defining parameter range
param_grid = {'kernel': ['rbf'], 'C': [1.0, 2.0, 10, 20],
              'kernel': ['linear'], 'C': [1.0, 2.0, 10, 20], 
              'gamma': [0.01, 0.02, 0.1, 0.2, 1.0, 2.0],} 

  
grid = GridSearchCV(SVR(), param_grid,cv=3 ,refit = True, verbose = 3)
  
# fitting the model for grid search
grid.fit(X_train, y_train)

Fitting 3 folds for each of 24 candidates, totalling 72 fits
[CV 1/3] END ..C=1.0, gamma=0.01, kernel=linear;, score=0.037 total time=  39.5s
[CV 2/3] END .C=1.0, gamma=0.01, kernel=linear;, score=-0.503 total time=  36.3s
[CV 3/3] END ..C=1.0, gamma=0.01, kernel=linear;, score=0.022 total time= 1.5min
[CV 1/3] END ..C=1.0, gamma=0.02, kernel=linear;, score=0.037 total time=  39.7s
[CV 2/3] END .C=1.0, gamma=0.02, kernel=linear;, score=-0.503 total time=  36.4s
[CV 3/3] END ..C=1.0, gamma=0.02, kernel=linear;, score=0.022 total time= 1.5min
[CV 1/3] END ...C=1.0, gamma=0.1, kernel=linear;, score=0.037 total time=  39.6s
[CV 2/3] END ..C=1.0, gamma=0.1, kernel=linear;, score=-0.503 total time=  34.7s
[CV 3/3] END ...C=1.0, gamma=0.1, kernel=linear;, score=0.022 total time= 1.5min
[CV 1/3] END ...C=1.0, gamma=0.2, kernel=linear;, score=0.037 total time=  39.6s
[CV 2/3] END ..C=1.0, gamma=0.2, kernel=linear;, score=-0.503 total time=  36.4s
[CV 3/3] END ...C=1.0, gamma=0.2, kernel=linear;

GridSearchCV(cv=3, estimator=SVR(),
             param_grid={'C': [1.0, 2.0, 10, 20],
                         'gamma': [0.01, 0.02, 0.1, 0.2, 1.0, 2.0],
                         'kernel': ['linear']},
             verbose=3)

In [55]:
# print best parameter after tuning
print(grid.best_params_)

{'C': 1.0, 'gamma': 0.01, 'kernel': 'linear'}
